# Generalized Cellular Automata Test
The goal of this project is to experiment with a new paradigm for generalized intelligence, inspired heavily by [this](https://www.youtube.com/watch?v=p-OYPRhqRCg) talk between Joscha Bach and Michael Levin. The ideas I'm drawing upon are primarily Bachs, however I take heavy inspiration by Levins work on collective intelligence in all forms of cells as well, seeking originally to make one based mostly on the idea of bioelectric signaling, and slowly shifting more towards Joscha Bachs idea on a search for selector_functions to find appropraite neighbors in higher dimensions to process information over long distances.

In [2]:
import os
import imageio
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import csv

In [3]:
def average_activation(neighbors):
    if neighbors:
        return sum(neighbor.state for neighbor in neighbors) / len(neighbors)
    return 0

def inverted_average(neighbors):
    if neighbors:
        return 1-(sum(neighbor.state for neighbor in neighbors) / len(neighbors))
    return 0

def sigmoid_activation(neighbors):
    total = sum(neighbor.state for neighbor in neighbors) if neighbors else 0
    return 1 / (1 + np.exp(-total))

def min_activation(neighbors):
    if neighbors:
        return min(neighbor.state for neighbor in neighbors)
    return 0

def max_activation(neighbors):
    if neighbors:
        return max(neighbor.state for neighbor in neighbors)
    return 0

def tanh_activation(neighbors):
    total = sum(neighbor.state for neighbor in neighbors) if neighbors else 0
    return (np.tanh(total) + 1) / 2  # Adjusting to [0, 1] range

def random_activation(neighbors):
    return (np.random.rand())

In [5]:
def save_grid_data(grid, step, save_dir):
    with open(f'{save_dir}/grid_data.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        for row in grid:
            writer.writerow([step] + [cell.state for cell in row])



def save_grid_image(grid, step, save_dir):
    image = np.array([[cell.state for cell in row] for row in grid])
    plt.imshow(image, cmap='gray')
    plt.title(f'Step {step}')
    plt.savefig(f'{save_dir}/step_{step}.png')
    plt.close()

def create_images_from_csv(save_dir):
    with open(f'{save_dir}/grid_data.csv', 'r') as file:
        reader = csv.reader(file)
        current_step = -1
        grid = []

        for row in reader:
            step = int(row[0])
            if step != current_step:
                if current_step != -1:
                    # Save the previous step's image
                    grid_array = np.array(grid)
                    plt.imshow(grid_array, cmap='gray')
                    plt.title(f'Step {current_step}')
                    plt.savefig(f'{save_dir}/step_{current_step}.png')
                    plt.close()

                current_step = step
                grid = []

            grid.append([float(cell) for cell in row[1:]])

        # Save the last step's image
        if grid:
            grid_array = np.array(grid)
            plt.imshow(grid_array, cmap='gray')
            plt.title(f'Step {current_step}')
            plt.savefig(f'{save_dir}/step_{current_step}.png')
            plt.close()

def create_gif(image_folder, gif_name):
    images = []

    def sort_key(file_name):
        numbers = re.findall(r'\d+', file_name)
        return int(numbers[0]) if numbers else 0

    file_names = sorted(os.listdir(image_folder), key=sort_key)

    for file_name in file_names:
        if file_name.endswith('.png'):
            file_path = os.path.join(image_folder, file_name)
            images.append(imageio.imread(file_path))

    imageio.mimsave(os.path.join(image_folder, gif_name), images, fps=10)

    for file_name in file_names:
            if file_name.endswith('.png'):
                file_path = os.path.join(image_folder, file_name)
                os.remove(file_path)

In [7]:
# Create a unique directory name based on the current date and time
current_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
experiment_dir = f'Experiments/Experiment_2/Experiment_{current_time}'
os.makedirs(experiment_dir, exist_ok=True)

class Cell:
    def __init__(self, x, y, activation_function, num_neighbors=4):
        self.x = x
        self.y = y
        self.state = 0.0
        self.reward = 0.0
        self.memory = []
        self.neighbors = []
        self.num_neighbors = num_neighbors
        self.connection_strength = {neighbor: 0.5 for neighbor in self.neighbors}
        self.activation_function = activation_function

    def update_selector(self, update_cells=None):
        # If no specific cells are provided, update all
        if update_cells is None:
            update_cells = range(self.num_neighbors)

        # Update the specified neighbors
        for i in update_cells:
            # Randomly select a new neighbor
            new_neighbor_x = np.random.randint(0, grid_size[0])
            new_neighbor_y = np.random.randint(0, grid_size[1])
            new_neighbor = (new_neighbor_x, new_neighbor_y)

            # Replace the old neighbor with the new one
            if i < len(self.neighbors):
                self.neighbors[i] = new_neighbor
            else:
                self.neighbors.append(new_neighbor)

            # Initialize connection strength for the new neighbor
            self.connection_strength[new_neighbor] = 0.5

    def update_state(self, new_state, memory):
        self.state = new_state
        self.memory.append(memory)

    def get_recent_memory(self):
        return self.memory[-10:]  # Return the last 10 states

# Function to get neighboring cells based on selector
def get_neighbors(cell):
    neighbors = []
    # for di in range(-1, 2):
    #     for dj in range(-1, 2):
    #         if di == 0 and dj == 0:
    #             continue  # Skip the cell itself
    #         ni, nj = cell.x + di, cell.y + dj
    #         if 0 <= ni < grid_size[0] and 0 <= nj < grid_size[1] and cell.selector[ni][nj]:
    #             neighbors.append(ca_grid[ni][nj])
    for i in cell.neighbors:
        neighbors.append(ca_grid[i[0], i[1]])

    return neighbors

# Define the cell update logic for continuous values
def update_cell_state(cell, neighbors):
    new_state = cell.activation_function(neighbors)
    new_reward = new_state

    # Create an array to hold zneighbor data
    # neighbors_data = []
    # for neighbor in neighbors:
    #     # Extract x, y coordinates and state of each neighbor
    #     neighbor_data = [neighbor.x, neighbor.y, neighbor.state, neighbor.reward]
    #     neighbors_data.append(neighbor_data)

    # # Save the new state and the neighbor data in memory
    # memory = [[new_state, new_reward], neighbors_data]
    memory = [new_state, new_reward]
    cell.update_state(new_state, memory)

# Define your activation functions and their weights
activation_functions = [average_activation, inverted_average, sigmoid_activation, min_activation, max_activation, tanh_activation, random_activation]
weights = [1/6, 1/6, 1/6, 1/6, 1/6, 1/6, 0] #[0.4, 0.1, 0.07, 0.16, 0.16, 0.07, 0.04]  # Adjust these weights as needed


# Learning mechanism
def apply_learning(cell):
    average_rewards = sum([m[1] for m in cell.get_recent_memory()]) / len(cell.get_recent_memory())

    if average_rewards < 0.2:
        chosen_function = np.random.choice(activation_functions, p=weights)
        cell.activation_function = chosen_function
        cell.update_selector()

    if average_rewards > 0.2 and average_rewards < 0.85:  # Low average reward
        neighbors = get_neighbors(cell)

        # Find the neighbor with the lowest sum of states over the last 10 memories
        lowest_sum_neighbor = None
        lowest_sum = float('inf')

        for neighbor in neighbors:
            neighbor_memory = neighbor.get_recent_memory()
            sum_states = sum([m[1] for m in neighbor_memory])  # Sum of rewards

            if sum_states < lowest_sum:
                lowest_sum = sum_states
                lowest_sum_neighbor = neighbor

        # If a neighbor with the lowest sum is found, update its selector in the current cell
        if lowest_sum_neighbor:
            index_of_neighbor = cell.neighbors.index((lowest_sum_neighbor.x, lowest_sum_neighbor.y))
            cell.update_selector(update_cells=[index_of_neighbor])
        
# Initialize the CA grid with Cell objects
grid_size = (10, 10)
ca_grid = np.empty(grid_size, dtype=object)
cycle_length = 5
for i in range(grid_size[0]):
    for j in range(grid_size[1]):
        chosen_function = np.random.choice(activation_functions, p=weights)
        ca_grid[i, j] = Cell(i, j, chosen_function)
        ca_grid[i, j].state = np.random.rand()
        ca_grid[i, j].update_selector()

# Main simulation loop
simulation_steps = 250
for step in range(simulation_steps):
    # Update each cell based on rules
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            cell = ca_grid[i, j]
            neighbors = get_neighbors(cell)
            update_cell_state(cell, neighbors)
            if step % cycle_length == 0:
                apply_learning(cell)
    print(step)
    
    # Save as image for visualization
    save_grid_data(ca_grid, step, experiment_dir)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
create_images_from_csv(experiment_dir)
create_gif(experiment_dir, 'ca_simulation.gif')

C:\Users\CYBER\AppData\Local\Temp\ipykernel_5928\3792796118.py:58: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


In [6]:
create_gif('Experiments/Experiment_2/Experiment_2023-12-01-02-32-16', 'ca_simulation.gif')

C:\Users\CYBER\AppData\Local\Temp\ipykernel_5928\3792796118.py:58: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


# Random ideas
* Now that we have the cellular automata engine, it's time to make it think. The basic idea is to have it's selector function decide which neighbors it looks for, probably up to the maximum of 8 it coulde have around it spatially. We can intialize them entirely randomly, or simply start with it's base neighbors.
* To get useful inputs and outputs, we need to have certain neurons represent inputs, and others represent outputs. For instance, we could encode some information in the continuous value of input neurons, with more neurons used for more complex information, and have the available actions mapped to certain outputs (ideally these outputs would be more arbitary and less hard-coded somehow in the future).
* The learning mechanism will be reward driven using an economic distribution sort of appraoch similar to the suggestions given by Joscha Bach in the Generalist AI talk between Joscha Bach and Michael Levin. We will attempt to track how much each neuron contributes to the final reward.
* Input neurons would be rewarded more for keeping the input information to encourage the system to keep the information.
* A basic training test we can imagine is a simple addition algorithm. For instance, we can create a simple 4 bit adder. To do so, we will have 2 sets of 4 cell inputs, and one set of a 4 bit output with a carry output.
* If a cell is negatively contributing to the loss function (or whatever metric we use to evaluate the result), cells will weaken their connection to it and eventually seek new neighbors, and if they're positively contributing cells will strengthen their connection. This can draw on the cells memory in some fashion as well.
* If a cell is itself negatively contributing to the loss function, it will randomly seek new neighbors in a hope to eventually contribute.
* Unlike a traditional neural net which would be trained and then ran in inference, the training of this automata would be continuously ongoing. I'll need to add ways to interact with it live to help test how resistant it is to perturbations over time and learning entirely new tasks within the network, for isntance starting with a 2 bit adder, upping it to 4, upping that to 8, etc.
* Since information would take time to propagate through the network and calculate (unless there was somehow direct connections to the task, but in practice the tasks should be too complex for such simple connections), the method by which rewards are propagated through the network should probably be analyzed every x steps or something of the sort. The memory could in theory play some sort of part in this, and in general it seems to line up with my recent thoughts on the importance of brain-wave like patterns in intelligence in general. This could in practice mean we can give a new input and read the output every x steps, say 10, making the network have an effective processing time of x steps * time per step.